In [36]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import os
import json
import requests 

This section will make API call and create dataframe with needed data from the Urban Institute of Education Portal accessing the Directory of Common Core of Data from the US Department of Education which is the primary database on public elementary and secondary education. Specifically pulling 2014 directory data for school districts in the state of Florida which based on the api documentation the state code is found under the fips search key at https://educationdata.urban.org/documentation/school-districts.html#overview

In [38]:
# Build query URL
url = "https://educationdata.urban.org/api/v1/school-districts/ccd/directory/"
year = "2014/"
state_code = "12"

query_url = f"{url}{year}?fips={state_code}"

# Request data in json and store variable
fl_ccd_directory_response = requests.get(query_url)
fl_ccd_directory_data = fl_ccd_directory_response.json()

# Establish the results from the request to loop through below
results_directory = fl_ccd_directory_data['results']

# Create Lists to store values for the columns
district_name = []
dir_nces_id = []
state_id = []
latitude = []
longitude = []
urban_locale = []
agency_type = []
total_schools = []
total_students = []
spec_ed_stud = []
eng_lang_stud = []

# Loop through results and store values in appropriate lists
for result in results_directory:
    district_name.append(result['lea_name'])
    dir_nces_id.append(result['leaid'])
    state_id.append(result['state_leaid'])
    latitude.append(result['latitude'])
    longitude.append(result['longitude'])
    urban_locale.append(result['urban_centric_locale'])
    agency_type.append(result['agency_type'])
    total_schools.append(result['number_of_schools'])
    total_students.append(result['enrollment'])
    spec_ed_stud.append(result['spec_ed_students'])
    eng_lang_stud.append(result['english_language_learners'])
    
# Create dataframe from results
fl_ccd_directory = pd.DataFrame({"Disctrict Name":district_name, "NCES ID":nces_id, "State ID":state_id,                          
                                 "Degree of urbanization":urban_locale, "Agency Type":agency_type,
                                "Total Schools":total_schools, "Total Students Enrolled":total_students,
                                "Special Ed Students":spec_ed_stud, "English Language Students":eng_lang_stud,
                                "Latitude":latitude, "Longitude":longitude})

fl_ccd_directory

,Disctrict Name,NCES ID,State ID,Degree of urbanization,Agency Type,Total Schools,Total Students Enrolled,Special Ed Students,English Language Students,Latitude,Longitude
0,FL VIRTUAL,1200002,71,12,8,5,6108,207,11,28.518801,-81.468399
1,CESA,1200005,76,12,5,0,-2,0,0,30.425200,-84.249298
2,ALACHUA,1200030,01,12,1,73,28689,3788,584,29.652601,-82.318977
3,BAKER,1200060,02,32,1,9,4936,538,11,30.277151,-82.118149
4,LAKE WALES CHARTER SCHOOLS,1200080,53D,22,7,0,-1,340,-1,27.901272,-81.588242
...,...,...,...,...,...,...,...,...,...,...,...
71,FAU LAB SCH,1202012,72,12,8,2,2420,185,73,26.373400,-80.109039
72,FSU LAB SCH,1202013,73,41,8,3,2411,166,35,30.385225,-84.223892
73,FAMU LAB SCH,1202014,74,12,8,2,482,20,0,30.416599,-84.286201
74,UF LAB SCH,1202015,75,12,8,2,1153,64,0,29.640368,-82.335220


This section will make API call and create dataframe with needed data from the Urban Institute of Education Portal accessing the Directory of Common Core of Data from the US Department of Education which is the primary database on public elementary and secondary education. Specifically pulling 2014 financal data for school districts in the state of Florida which based on the api documentation the state code is found under the fips search key at https://educationdata.urban.org/documentation/school-districts.html#overview

In [58]:
# Build query URL
url = "https://educationdata.urban.org/api/v1/school-districts/ccd/finance/"
year = "2014/"
state_code = "12"

query_url = f"{url}{year}?fips={state_code}"

# Request data in json and store variable
fl_ccd_finance_response = requests.get(query_url)
fl_ccd_finance_data = fl_ccd_finance_response.json()

# Establish the results from the request to loop through below
results_financial = fl_ccd_finance_data['results']

# Create Lists to store values for the columns
fin_nces_id = []
total_rev = []
fed_total_rev = []
st_total_rev = []
loc_total_rev = []
loc_prop_tax_rev = []
intruct_staff_exp = []
general_admin_exp = []
school_admin_exp = []
operation_plant_exp = []
student_transport_exp = []

# Loop through results and store values in appropriate lists
for result in results_financial:
    fin_nces_id.append(result['leaid'])
    total_rev.append(result['rev_total'])
    fed_total_rev.append(result['rev_fed_total'])
    st_total_rev.append(result['rev_state_total'])
    loc_total_rev.append(result['rev_local_total'])
    loc_prop_tax_rev.append(result['rev_local_prop_tax'])
    intruct_staff_exp.append(result['exp_current_instruc_staff'])
    general_admin_exp.append(result['exp_current_general_admin'])
    school_admin_exp.append(result['exp_current_sch_admin'])
    operation_plant_exp.append(result['exp_current_operation_plant'])
    student_transport_exp.append(result['exp_current_student_transport'])
    
# Create dataframe from results
fl_ccd_financial = pd.DataFrame({"NCES ID":nces_id, "Total Revenue":total_rev, "Total Federal Revenue":fed_total_rev,                        
                                 "Total State Revenue":st_total_rev, "Total Local Revenue":loc_total_rev,
                                "Local Revenue (Property Tax)":loc_prop_tax_rev, "Total Expense for Instruction Staff":intruct_staff_exp,
                                "Total Expense for General Admin":general_admin_exp, "Total Expense for School Admin":school_admin_exp,
                                "Total Expense for Operations":operation_plant_exp, "Total Expense for Student Transportation":student_transport_exp})

fl_ccd_financial

,NCES ID,Total Revenue,Total Federal Revenue,Total State Revenue,Total Local Revenue,Local Revenue (Property Tax),Total Expense for Instruction Staff,Total Expense for General Admin,Total Expense for School Admin,Total Expense for Operations,Total Expense for Student Transportation
0,1200002,-1.0,-1.0,-1.0,-1,-2,-1,-1,-1,-1,-1
1,1200005,-2.0,-2.0,-2.0,-2,-2,-2,-2,-2,-2,-2
2,1200030,272451000.0,34881000.0,119810000.0,117760000,103083000,21763000,3041000,13128000,29535000,11174000
3,1200060,42674000.0,4704000.0,29197000.0,8773000,6266000,2595000,732000,2007000,4385000,2613000
4,1200080,-1.0,-1.0,-1.0,-1,-2,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
71,1202012,-1.0,-1.0,-1.0,-1,-2,-1,-1,-1,-1,-1
72,1202013,-1.0,-1.0,-1.0,-1,-2,-1,-1,-1,-1,-1
73,1202014,-1.0,-1.0,-1.0,-1,-2,-1,-1,-1,-1,-1
74,1202015,-1.0,-1.0,-1.0,-1,-2,-1,-1,-1,-1,-1


This section will make API call from the Urban Institute of Education Portal accessing The US Census Bureau's Small Area Income and Poverty Estimates.  Specifically pulling 2014 estimated poverity data for school aged children fo school districts in the state of Florida.  Use the same fips paramaters as above.

In [19]:
# Build query URL
url = "https://educationdata.urban.org/api/v1/school-districts/saipe/"
year = "2014/"
state_code = "12"

query_url = f"{url}{year}?fips={state_code}"

# Request data in json and store variable
fl_ccd_poverty_response = requests.get(query_url)
fl_ccd_poverty_data = fl_ccd_poverty_response.json()

This section will read in the excel files from the Florida Department of Education for 2014.

In [35]:
# Read in Florida District 21+ Absents Data
fl_2014_21absents_path = "fldoe_data/2014_FL_21absents.xlsx"
fl_2014_21absents_data = pd.read_excel(os.path.join(fl_2014_21absents_path), header=2)

fl_2014_21absents_data.head()

,District #,District Name,Enrollments,Absent 21 Days or Over,% Absent 21 or More Days
0,0,FLORIDA,3111840,303913,0.097663
1,1,ALACHUA,32688,3101,0.094867
2,2,BAKER,5377,1005,0.186907
3,3,BAY,32074,4095,0.127674
4,4,BRADFORD,3820,703,0.184031


In [36]:
# Read in Florida District Dropout Data
fl_2014_dropout_path = "fldoe_data/2014_FL_dropout_rates.xls"
fl_2014_dropout_data = pd.read_excel(os.path.join(fl_2014_dropout_path),header=4)

fl_2014_dropout_data.head()

,Dist #,District Name,White Dropouts (numerator),White Enrollment (denominator),White Dropout Rate,Black Dropouts (numerator),Black Enrollment (denominator),Black Dropout Rate,Hispanic Dropouts (numerator),Hispanic Enrollment (denominator),...,Pacific Islander Dropout Rate,American Indian Dropouts (numerator),American Indian Enrollment (denominator),American Indian Dropout Rate,Two or More Races Dropouts (numerator),Two or More Races Enrollment (denominator),Two or More Races Dropout Rate,Total Dropouts (numerator),Total Enrollment (denominator),Total Dropout Rate
0,0,FLORIDA,4858,365560,0.0132892,5430,201234,0.026984,4753,258350,...,0.0164569,73,3586,0.0203569,355,24532,0.0144709,15607,877251,0.017791
1,1,ALACHUA,58,4418,0.0131281,174,3115,0.055859,17,803,...,*,1,20,0.05,14,419,0.0334129,265,9182,0.028861
2,2,BAKER,13,1131,0.0114943,0,151,0.000000,0,24,...,*,*,*,*,0,26,0,13,1344,0.009673
3,3,BAY,65,5713,0.0113776,16,1255,0.012749,8,461,...,*,0,34,0,4,359,0.0111421,95,8033,0.011826
4,4,BRADFORD,2,686,0.00291545,4,210,0.019048,0,24,...,*,*,*,*,0,24,0,6,955,0.006283


In [37]:
# Read in Florida District Demographic Data
fl_2014_demographics_path = "fldoe_data/2014_FL_membership_demographics.xls"
fl_2014_demographics_data = pd.read_excel(os.path.join(fl_2014_demographics_path), sheet_name = "DISTRICT", header=4)

fl_2014_demographics_data.head()

,District #,District,Grade,White,Black or African American,Hispanic/Latino,Asian,Native Hawaiian or Other Pacific Islander,American Indian or Alaska Native,Two or More Races,Female Total,Male Total,Total
0,1,ALACHUA,PK,161,577,70,16,1,6,46,380,497,877
1,1,ALACHUA,KG,967,862,198,111,1,2,176,1082,1235,2317
2,1,ALACHUA,1,1042,843,215,103,3,4,180,1136,1254,2390
3,1,ALACHUA,2,970,847,228,134,,1,171,1180,1171,2351
4,1,ALACHUA,3,930,862,200,107,2,1,160,1076,1186,2262


In [38]:
# Read in Florida District Teacher Out of Field Data
fl_2014_teacher_outfield_path = "fldoe_data/2014_FL_teacher_outoffield.xls"
fl_2014_teacher_outfield_data = pd.read_excel(os.path.join(fl_2014_teacher_outfield_path), header=3)

fl_2014_teacher_outfield_data.head()

,Dist #,District Name,Sch #,School Name,Total\nCourses,Total\nIn Field,Total\nOut Field,% In Field,% Out Field
0,0,FLORIDA,,STATE TOTAL,3232752,3042970,189782,0.94129,0.05871
1,1,ALACHUA,,DISTRICT TOTAL,34441,34302,139,0.99596,0.00404
2,1,ALACHUA,0021,CHARLES W. DUVAL ELEMENTARY SCHOOL,499,499,0,1.00000,0.00000
3,1,ALACHUA,0031,J. J. FINLEY ELEMENTARY SCHOOL,762,762,0,1.00000,0.00000
4,1,ALACHUA,0041,STEPHEN FOSTER ELEMENTARY SCHOOL,758,758,0,1.00000,0.00000


In [44]:
# Read in Florida District Free and Reduced Lunch Data - specific sheet read is the latest survey for the year
#2014-15 SURVEY 3 PRELIMINARY DATA (as of 5/5/2015)
fl_2014_lunch_path = "fldoe_data/2014_FL_lunch_status.xlsx"
fl_2014_lunch_data = pd.read_excel(os.path.join(fl_2014_lunch_path), sheet_name = "Demo Preliminary Survey 3-Dist",
                                  header=4)

fl_2014_lunch_data.head()

,District Number,District Name,Grade,White,Unnamed: 4,Unnamed: 5,Black or African American,Unnamed: 7,Unnamed: 8,Hispanic or Latino,...,Unnamed: 17,American Indian or Alaska Native,Unnamed: 19,Unnamed: 20,Two or More Races,Unnamed: 22,Unnamed: 23,Total,Unnamed: 25,Unnamed: 26
0,NaN,NaN,NaN,Female,Male,Total,Female,Male,Total,Female,...,Total,Female,Male,Total,Female,Male,Total,Female,Male,Total
1,1.0,ALACHUA,PK,47,67,114,255,315,570,21,...,1,3,3,6,24,23,47,351,454,805
2,1.0,ALACHUA,KG,155,180,335,331,414,745,47,...,1,0,1,1,47,53,100,588,741,1329
3,1.0,ALACHUA,1,154,191,345,351,365,716,58,...,2,1,1,2,59,59,118,632,706,1338
4,1.0,ALACHUA,2,167,161,328,344,345,689,55,...,0,1,0,1,55,55,110,641,637,1278
